In [ ]:
import os
import json
import requests
import jsonlines
from tqdm import tqdm
import time
from multiprocessing import Pool
from functools import partial

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    RetryError
)

In [6]:
os.environ['WENXIN_KEY'] = "your wenxin key"
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(5))
def ernie_bot_api(**kwargs):
    try:
        if not os.environ.get('WENXIN_KEY'):
            print(f"Environment variable 'WENXIN_KEY' not exists. Please set")
            return
        url = f"your wenxin query url"
        payload = json.dumps({
            "messages": kwargs["messages"],
            "system": kwargs.get("system", None),
        })
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        if kwargs.get('stream', False):
            result = []
            for line in response.iter_lines():
                content = line.decode("utf-8")[5:]
                if len(content.strip()) == 0:
                    continue
                result.append(json.loads(content)['result'])
            generation = "".join(result)
        else:
            generation = json.loads(response.text)['result']
        return generation
    except Exception as e:
        print(response.text)

In [3]:
def process(item):
    message = []
    for prompt in item['prompts']:
        message.append({"role": "user", "content": prompt})
        answer = ernie_bot_api(messages=message, system=item['system'])
        message.append({"role": "assistant", "content": answer})
    message = [{"role": "system", "content": item['system']}] + message
    item['messages'] = message
    return item

In [5]:
data = list(jsonlines.open("../data/wiki_roleplay_multilingual_test_input_w_evidence.jsonl"))

In [ ]:
results = []
pbar = tqdm(total=len(data))
for item in data:
    pbar.update(1)
    results.append(process(item))

In [ ]:
with jsonlines.open(f"../data/results/wiki_roleplay_multilingual_test_input_w_evidence_wenxin_output.jsonl", "w") as f:
    for each in results:
        f.write(each)